#### Download heart disease dataset heart.csv in Exercise folder and do following, (credits of dataset: https://www.kaggle.com/fedesoriano/heart-failure-prediction)

1. Load heart disease dataset in pandas dataframe
2. Remove outliers using Z score. Usual guideline is to remove anything that has Z score > 3 formula or Z score < -3
3. Convert text columns to numbers using label encoding and one hot encoding
4. Apply scaling
5. Build a classification model using support vector machine. Use standalone model as well as Bagging model and check if you see any difference in the performance.
6. Now use decision tree classifier. Use standalone model as well as Bagging and check if you notice any difference in performance
7. Comparing performance of svm and decision tree classifier figure out where it makes most sense to use bagging and why. Use internet to figure out in what conditions bagging works the best.

In [2]:
import pandas as pd
df = pd.read_csv('heart.csv')
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [3]:
df.shape

(918, 12)

In [4]:
df.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


In [5]:
df.isnull().sum()

Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64

In [6]:
df['ChestPainType'].unique()

array(['ATA', 'NAP', 'ASY', 'TA'], dtype=object)

In [7]:
df = df[df.RestingBP <= (df.RestingBP.mean() + (3* df.RestingBP.std()))]
df.shape

(911, 12)

In [8]:
df = df[df.Cholesterol <= (df.Cholesterol.mean() + (3 * df.Cholesterol.std()))]
df.shape

(908, 12)

In [9]:
df[df.Oldpeak > (df.Oldpeak.mean() + (3 * df.Oldpeak.std()))]

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
166,50,M,ASY,140,231,0,ST,140,Y,5.0,Flat,1
702,59,M,TA,178,270,0,LVH,145,N,4.2,Down,0
771,55,M,ASY,140,217,0,Normal,111,Y,5.6,Down,1
791,51,M,ASY,140,298,0,Normal,122,Y,4.2,Flat,1
850,62,F,ASY,160,164,0,LVH,145,N,6.2,Down,1
900,58,M,ASY,114,318,0,ST,140,N,4.4,Down,1


In [10]:
df = df[df.Oldpeak <= (df.Oldpeak.mean() + (3 * df.Oldpeak.std()))]
df.shape

(902, 12)

In [12]:
df.RestingECG.unique()

array(['Normal', 'ST', 'LVH'], dtype=object)

In [13]:
df.ST_Slope.unique()

array(['Up', 'Flat', 'Down'], dtype=object)

In [16]:
df.RestingECG.replace({
    'Normal': 1,
    'ST': 2,
    'LVH': 3
}, inplace=True)
df.ExerciseAngina.replace({
    'N': 0,
    'Y': 1
}, inplace=True)
df.ST_Slope.replace({
    'Down': 0,
    'Flat': 1,
    'Up': 2
}, inplace=True)
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,1,172,0,0.0,2,0
1,49,F,NAP,160,180,0,1,156,0,1.0,1,1
2,37,M,ATA,130,283,0,2,98,0,0.0,2,0
3,48,F,ASY,138,214,0,1,108,1,1.5,1,1
4,54,M,NAP,150,195,0,1,122,0,0.0,2,0


In [26]:
df = pd.get_dummies(df, drop_first=True)

In [28]:
X = df.drop(['HeartDisease'], axis='columns')
y = df.HeartDisease

In [31]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [34]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

cross_val_score(DecisionTreeClassifier(), X_scaled, y, cv=5).mean()

0.7282872928176796

In [36]:
from sklearn.svm import SVC
cross_val_score(SVC(), X_scaled, y, cv=5).mean()

0.8159054634745242

In [41]:
from sklearn.ensemble import BaggingClassifier

model = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(random_state=0), 
    n_estimators=100, 
    max_samples=0.9, 
    oob_score=True,
    random_state=0
)
model.fit(X_scaled, y)
model.oob_score_

0.8580931263858093

In [42]:
model = BaggingClassifier(
    base_estimator=SVC(random_state=0), 
    n_estimators=100, 
    max_samples=0.9, 
    oob_score=True,
    random_state=0
)
model.fit(X_scaled, y)
model.oob_score_

0.8614190687361419